In [1]:
import numpy as np
import operator
from os import listdir


In [2]:
def classify0(inX, dataSet, labels, k):

    dataSetSize = dataSet.shape[0]

    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet

    sqDiffMat = diffMat**2

    sqDistances = sqDiffMat.sum(axis=1)

    distances = sqDistances**0.5

    sortedDistIndices = distances.argsort()

    classCount = {}
    for i in range(k):

        voteIlabel = labels[sortedDistIndices[i]]

        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1

    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)

    return sortedClassCount[0][0]

In [3]:
def img2vector(filename):

    returnVect = np.zeros((1, 1024))

    fr = open(filename)

    for i in range(32):

        lineStr = fr.readline()

        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])

    return returnVect

In [4]:
hwLabels = []

trainingFileList = listdir('/data/datasets/chengkun/homework/trainingDigits')

m = len(trainingFileList)

trainingMat = np.zeros((m, 1024))

for i in range(m):

    fileNameStr = trainingFileList[i]

    classNumber = int(fileNameStr.split('_')[0])

    hwLabels.append(classNumber)

    trainingMat[i,:] = img2vector('/data/datasets/chengkun/homework/trainingDigits/%s' % (fileNameStr))


    
tsLabels = []

testFileList = listdir('/data/datasets/chengkun/homework/testDigits')

n = len(testFileList)

testMat = np.zeros((n, 1024))

for i in range(n):

    fileNameStr = testFileList[i]

    classNumber = int(fileNameStr.split('_')[0])

    tsLabels.append(classNumber)

    testMat[i,:] = img2vector('/data/datasets/chengkun/homework/testDigits/%s' % (fileNameStr))

In [5]:
print(testMat)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
len(trainingMat)

1934

In [7]:
print(tsLabels[1])

9


In [8]:
print(len(tsLabels))

946


In [9]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from pathlib import Path

import datetime
import time
import os
import sys
import re 
from collections import Counter, defaultdict
from concurrent.futures import ProcessPoolExecutor, wait
from functools import partial
import mmap
import json
import pickle as pkl
import gc
import logging
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, cross_val_predict, train_test_split,GridSearchCV
from sklearn.metrics import roc_auc_score,recall_score, average_precision_score, roc_curve,f1_score, auc, precision_recall_fscore_support,precision_recall_curve,accuracy_score,precision_score,mean_squared_error,mean_squared_error,mean_absolute_percentage_error
from xgboost.sklearn import XGBClassifier
import math
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.decomposition import PCA
from tqdm import tqdm
import shap

import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

CPU_COUNT=9
from sklearn.feature_selection import VarianceThreshold
import xgboost as xgb

## SVM

#### Linear Kernel

In [10]:
i = 0

In [11]:
def SVM_model(C,max_iter,tol):
    
    
    
    svm_model = svm.SVC(
        max_iter= int (max_iter),
                                    tol =  tol,
                                C= C,
                       probability = True,
                        kernel = 'linear'
                       )
    
    val = cross_val_score(svm_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()

    return val


def tune( tasks, nb, nit, model_name, param_tune):
    for task in tasks:
        print(f"current task: {task} {model_name}")
        #res_output = f"{task}year_{model_name}.txt"
        #model_dump = f"{task}year_{model_name}_model.pkl"
        


        
        BO = BayesianOptimization( f = SVM_model,pbounds = param_tune,verbose = 2,random_state = 1 )
        
        BO.maximize(n_iter=5)
        print(BO.max)
        return BO.max

#### Learning curve

In [12]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.75, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)


param_tune = {

        'max_iter': (1, 5000),
        'tol': (0.0001,1),
        'C': (1, 50)
                }
tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'SVM', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

svm_model = svm.SVC(
            max_iter= int (params['params']['max_iter']),
                                    tol =  params['params']['tol'],
                                C= params['params']['C'],
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


current task: 0 SVM
|   iter    |  target   |     C     | max_iter  |    tol    |
-------------------------------------------------------------
| 1         | 0.9524    | 21.43     | 3.602e+03 | 0.0002144 |
| 2         | 0.9483    | 15.81     | 734.6     | 0.09243   |
| 3         | 0.9545    | 10.13     | 1.728e+03 | 0.3968    |
| 4         | 0.9462    | 27.4      | 2.097e+03 | 0.6853    |
| 5         | 0.9524    | 11.02     | 4.391e+03 | 0.02748   |
| 6         | 0.94      | 12.4      | 4.391e+03 | 0.8158    |
| 7         | 0.9504    | 27.08     | 3.937e+03 | 0.175     |
| 8         | 0.94      | 30.2      | 1.306e+03 | 0.7468    |
| 9         | 0.9421    | 12.15     | 778.8     | 0.9473    |
| 10        | 0.9525    | 36.59     | 3.643e+03 | 0.1948    |
{'target': 0.9545317869415808, 'params': {'C': 10.126750357505875, 'max_iter': 1728.4580744881957, 'tol': 0.39682779748324687}}
Tuning:
3.0839550495147705
Training:
0.10509991645812988
Predicting:
0.0880281925201416
Test accuracy:  0.96

In [13]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.5, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)


param_tune = {

        'max_iter': (1, 5000),
        'tol': (0.0001,1),
        'C': (1, 50)
                }
tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'SVM', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

svm_model = svm.SVC(
            max_iter= int (params['params']['max_iter']),
                                    tol =  params['params']['tol'],
                                C= params['params']['C'],
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


current task: 0 SVM
|   iter    |  target   |     C     | max_iter  |    tol    |
-------------------------------------------------------------
| 1         | 0.9752    | 21.43     | 3.602e+03 | 0.0002144 |
| 2         | 0.9752    | 15.81     | 734.6     | 0.09243   |
| 3         | 0.9741    | 10.13     | 1.728e+03 | 0.3968    |
| 4         | 0.9731    | 27.4      | 2.097e+03 | 0.6853    |
| 5         | 0.9752    | 11.02     | 4.391e+03 | 0.02748   |
| 6         | 0.972     | 4.5       | 194.4     | 0.8406    |
| 7         | 0.9741    | 11.74     | 4.39e+03  | 0.3865    |
| 8         | 0.971     | 21.49     | 3.602e+03 | 0.6716    |
| 9         | 0.972     | 12.15     | 778.8     | 0.9473    |
| 10        | 0.9741    | 36.59     | 3.643e+03 | 0.1948    |
{'target': 0.975161583248758, 'params': {'C': 21.434078230426127, 'max_iter': 3601.9021427173484, 'tol': 0.00021436337986315216}}
Tuning:
4.269452810287476
Training:
0.5254440307617188
Predicting:
0.16532087326049805
Test accuracy:  0.9

In [14]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.25, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)


param_tune = {

        'max_iter': (1, 5000),
        'tol': (0.0001,1),
        'C': (1, 50)
                }
tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'SVM', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

svm_model = svm.SVC(
            max_iter= int (params['params']['max_iter']),
                                    tol =  params['params']['tol'],
                                C= params['params']['C'],
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


current task: 0 SVM
|   iter    |  target   |     C     | max_iter  |    tol    |
-------------------------------------------------------------
| 1         | 0.9766    | 21.43     | 3.602e+03 | 0.0002144 |
| 2         | 0.9766    | 15.81     | 734.6     | 0.09243   |
| 3         | 0.9759    | 10.13     | 1.728e+03 | 0.3968    |
| 4         | 0.9717    | 27.4      | 2.097e+03 | 0.6853    |
| 5         | 0.9766    | 11.02     | 4.391e+03 | 0.02748   |
| 6         | 0.9724    | 12.4      | 4.391e+03 | 0.8158    |
| 7         | 0.9772    | 27.08     | 3.937e+03 | 0.175     |
| 8         | 0.9731    | 30.2      | 1.306e+03 | 0.7468    |
| 9         | 0.9738    | 12.15     | 778.8     | 0.9473    |
| 10        | 0.9772    | 36.59     | 3.643e+03 | 0.1948    |
{'target': 0.9772413793103448, 'params': {'C': 27.082598095553447, 'max_iter': 3937.1326600057873, 'tol': 0.17498228480326983}}
Tuning:
7.626207590103149
Training:
0.8466277122497559
Predicting:
0.1896207332611084
Test accuracy:  0.9862

### Train on the whole dataset and tune the parameter individually

In [15]:

import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



svm_model = svm.SVC(
                                    tol =  0.00001,
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)


Training:
1.5295608043670654
Predicting:
0.2436506748199463
Test accuracy:  0.985200845665962


In [16]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



svm_model = svm.SVC(
                                    tol =  0.5,
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)


Training:
0.9948046207427979
Predicting:
0.18914508819580078
Test accuracy:  0.9841437632135307


In [17]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



svm_model = svm.SVC(
                                    tol =  1,
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)


Training:
0.7527580261230469
Predicting:
0.16050457954406738
Test accuracy:  0.9830866807610994


In [18]:
for para in [200,2000,5000]:


    tr_dx = np.array(trainingMat).astype(float)
    tr_dy = np.array(hwLabels).astype(float)
    ts_dx = np.array(testMat).astype(float)
    ts_dy = np.array(tsLabels).astype(float)



    svm_model = svm.SVC(
                                        max_iter =  para,
                           probability = True,
                            kernel = 'linear',decision_function_shape='ovr'
                           )


    tm_st = time.time() 
    svm_model.fit(tr_dx,tr_dy)
    tm_ed = time.time()
    print("Training:")
    print(tm_ed-tm_st)


    tm_st = time.time()        
    test_preds = svm_model.predict_proba(ts_dx)
    tm_ed = time.time()
    print("Predicting:")
    print(tm_ed-tm_st)


    preds_1 = test_preds.argmax(axis=1)
    ts_accuracy_score = accuracy_score(ts_dy, preds_1)
    print("Test accuracy: " ,ts_accuracy_score)


Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.


Training:
1.494490623474121
Predicting:
0.24400830268859863
Test accuracy:  0.9862579281183932
Training:
1.5026297569274902
Predicting:
0.243330717086792
Test accuracy:  0.9862579281183932
Training:
1.442300796508789
Predicting:
0.24682235717773438
Test accuracy:  0.9862579281183932


In [19]:
for para in [1,25,50]:


    tr_dx = np.array(trainingMat).astype(float)
    tr_dy = np.array(hwLabels).astype(float)
    ts_dx = np.array(testMat).astype(float)
    ts_dy = np.array(tsLabels).astype(float)



    svm_model = svm.SVC(
                                        C =  para,
                           probability = True,
                            kernel = 'linear',decision_function_shape='ovr'
                           )


    tm_st = time.time() 
    svm_model.fit(tr_dx,tr_dy)
    tm_ed = time.time()
    print("Training:")
    print(tm_ed-tm_st)


    tm_st = time.time()        
    test_preds = svm_model.predict_proba(ts_dx)
    tm_ed = time.time()
    print("Predicting:")
    print(tm_ed-tm_st)


    preds_1 = test_preds.argmax(axis=1)
    ts_accuracy_score = accuracy_score(ts_dy, preds_1)
    print("Test accuracy: " ,ts_accuracy_score)


Training:
1.4982478618621826
Predicting:
0.2429964542388916
Test accuracy:  0.985200845665962
Training:
1.4414925575256348
Predicting:
0.24317312240600586
Test accuracy:  0.985200845665962
Training:
1.4936692714691162
Predicting:
0.24358201026916504
Test accuracy:  0.9873150105708245


### the Bayes optimization

In [20]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)


param_tune = {

        'max_iter': (1, 5000),
        'tol': (0.0001,1),
        'C': (1, 50)
                }
tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'SVM', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

svm_model = svm.SVC(
            max_iter= int (params['params']['max_iter']),
                                    tol =  params['params']['tol'],
                                C= params['params']['C'],
                       probability = True,
                        kernel = 'linear',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)



val = cross_val_score(svm_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()        
print("5-fold Test accuracy: " ,val)        


current task: 0 SVM
|   iter    |  target   |     C     | max_iter  |    tol    |
-------------------------------------------------------------
| 1         | 0.9752    | 21.43     | 3.602e+03 | 0.0002144 |
| 2         | 0.9747    | 15.81     | 734.6     | 0.09243   |
| 3         | 0.9752    | 10.13     | 1.728e+03 | 0.3968    |
| 4         | 0.9741    | 27.4      | 2.097e+03 | 0.6853    |
| 5         | 0.9752    | 11.02     | 4.391e+03 | 0.02748   |
| 6         | 0.9731    | 12.4      | 4.391e+03 | 0.8158    |
| 7         | 0.9762    | 27.08     | 3.937e+03 | 0.175     |
| 8         | 0.9721    | 30.2      | 1.306e+03 | 0.7468    |
| 9         | 0.9726    | 12.15     | 778.8     | 0.9473    |
| 10        | 0.9767    | 36.59     | 3.643e+03 | 0.1948    |
{'target': 0.9767227644562263, 'params': {'C': 36.58654654217979, 'max_iter': 3643.3314572396093, 'tol': 0.19476856277911697}}
Tuning:
11.301583528518677
Training:
1.2195179462432861
Predicting:
0.22110891342163086
Test accuracy:  0.985

## Sigmoid Kernel

In [21]:
def SVM_model(C,max_iter,tol):
    
    
    
    svm_model = svm.SVC(
        max_iter= int (max_iter),
                                    tol =  tol,
                                C= C,
                       probability = True,
                        kernel = 'sigmoid'
                       )
    
    val = cross_val_score(svm_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()

    return val


def tune( tasks, nb, nit, model_name, param_tune):
    for task in tasks:
        print(f"current task: {task} {model_name}")
        #res_output = f"{task}year_{model_name}.txt"
        #model_dump = f"{task}year_{model_name}_model.pkl"
        


        
        BO = BayesianOptimization( f = SVM_model,pbounds = param_tune,verbose = 2,random_state = 1 )
        
        BO.maximize(n_iter=5)
        print(BO.max)
        return BO.max
    


In [22]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)


param_tune = {

        'max_iter': (1, 5000),
        'tol': (0.0001,1),
        'C': (1, 50)
                }
tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'SVM', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

svm_model = svm.SVC(
            max_iter= int (params['params']['max_iter']),
                                    tol =  params['params']['tol'],
                                C= params['params']['C'],
                       probability = True,
                        kernel = 'sigmoid',decision_function_shape='ovr'
                       )

        
tm_st = time.time() 
svm_model.fit(tr_dx,tr_dy)
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)


tm_st = time.time()        
test_preds = svm_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

val = cross_val_score(svm_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()        
print("5-fold Test accuracy: " ,val)        


current task: 0 SVM
|   iter    |  target   |     C     | max_iter  |    tol    |
-------------------------------------------------------------
| 1         | 0.9017    | 21.43     | 3.602e+03 | 0.0002144 |
| 2         | 0.9059    | 15.81     | 734.6     | 0.09243   |
| 3         | 0.9142    | 10.13     | 1.728e+03 | 0.3968    |
| 4         | 0.8971    | 27.4      | 2.097e+03 | 0.6853    |
| 5         | 0.9193    | 11.02     | 4.391e+03 | 0.02748   |
| 6         | 0.9131    | 12.4      | 4.391e+03 | 0.8158    |
| 7         | 0.9152    | 11.27     | 4.388e+03 | 0.7478    |
| 8         | 0.9214    | 7.438     | 4.389e+03 | 0.6423    |
| 9         | 0.9255    | 6.228     | 4.391e+03 | 0.2384    |
| 10        | 0.925     | 6.481     | 4.397e+03 | 0.621     |
{'target': 0.9255372133188737, 'params': {'C': 6.227538554955462, 'max_iter': 4391.3007169872735, 'tol': 0.23837596522068014}}
Tuning:
10.853391647338867
Training:
1.249006986618042
Predicting:
0.22066950798034668
Test accuracy:  0.9397

## XGBoost

In [23]:
def xgb_cv(n_estimators,gamma, max_depth,min_child_weight):
    
    
    
    xgb_model = XGBClassifier(max_depth = int(max_depth), 
                              n_estimators = int(n_estimators) ,
                              objective = 'binary:logistic',
                                booster = 'gbtree', 
                              gamma = min(gamma,1),
                              min_child_weight = min(min_child_weight,1),
                              random_state = 77)
                              #scale_pos_weight = min(scale_pos_weight,1),random_state = 77,decision_function_shape='ovr' )
    
    val = cross_val_score(xgb_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()

    return val


def tune( tasks, nb, nit, model_name, param_tune):
    for task in tasks:
        print(f"current task: {task} {model_name}")
        #res_output = f"{task}year_{model_name}.txt"
        #model_dump = f"{task}year_{model_name}_model.pkl"
        


        
        BO = BayesianOptimization( f = xgb_cv,pbounds = param_tune,verbose = 2,random_state = 777 )
        
        BO.maximize(n_iter=5)
        print(BO.max)
        return BO.max


### Learning curve

In [24]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.75, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {


            'max_depth' :(1,10),
            'n_estimators' :(1,50),

            'gamma': (0,1), 
            'min_child_weight' : (0,1)
                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

xgb_model = XGBClassifier(max_depth = int(params['params']['max_depth']),
                                  
                                  n_estimators = int(params['params']['n_estimators']), 
                                  objective = 'binary:logistic',
                                 booster = 'gbtree', 
                                  gamma = params['params']['gamma'], 
                                  min_child_weight = params['params']['min_child_weight'],
                                  #scale_pos_weight= params['params']['scale_pos_weight'],
                                 random_state = 77)

        
tm_st = time.time()
xgb_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = xgb_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


current task: 0 XG
|   iter    |  target   |   gamma   | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.8965    | 0.1527    | 3.721     | 0.06204   | 23.53     |
| 2         | 0.882     | 0.8353    | 9.343     | 0.727     | 38.66     |
| 3         | 0.8426    | 0.2692    | 6.796     | 0.09337   | 4.905     |
| 4         | 0.8902    | 0.5896    | 4.09      | 0.9889    | 31.7      |
| 5         | 0.88      | 0.6818    | 5.97      | 0.2689    | 19.29     |
| 6         | 0.8986    | 0.0       | 1.0       | 0.0       | 50.0      |
| 7         | 0.8923    | 1.0       | 10.0      | 1.0       | 50.0      |
| 8         | 0.8902    | 0.4111    | 4.263     | 0.7557    | 31.6      |
| 9         | 0.8903    | 0.0       | 1.0       | 0.0       | 42.42     |
| 10        | 0.8841    | 1.0       | 1.0       | 0.0       | 26.4      |
{'target': 0.8985609965635739, 'params': {'gamma': 0.0, 'max_depth': 1.0, 'min_child_weight':

In [25]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.5, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {


            'max_depth' :(1,10),
            'n_estimators' :(1,50),

            'gamma': (0,1), 
            'min_child_weight' : (0,1)
                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

xgb_model = XGBClassifier(max_depth = int(params['params']['max_depth']),
                                  
                                  n_estimators = int(params['params']['n_estimators']), 
                                  objective = 'binary:logistic',
                                 booster = 'gbtree', 
                                  gamma = params['params']['gamma'], 
                                  min_child_weight = params['params']['min_child_weight'],
                                  #scale_pos_weight= params['params']['scale_pos_weight'],
                                 random_state = 77)

        
tm_st = time.time()
xgb_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = xgb_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


current task: 0 XG
|   iter    |  target   |   gamma   | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9369    | 0.1527    | 3.721     | 0.06204   | 23.53     |
| 2         | 0.9255    | 0.8353    | 9.343     | 0.727     | 38.66     |
| 3         | 0.9027    | 0.2692    | 6.796     | 0.09337   | 4.905     |
| 4         | 0.9317    | 0.5896    | 4.09      | 0.9889    | 31.7      |
| 5         | 0.9379    | 0.6818    | 5.97      | 0.2689    | 19.29     |
| 6         | 0.9296    | 0.0       | 1.0       | 1.0       | 50.0      |
| 7         | 0.9234    | 1.0       | 10.0      | 1.0       | 24.58     |
| 8         | 0.8924    | 1.0       | 1.0       | 1.0       | 18.3      |
| 9         | 0.9338    | 0.6832    | 6.83      | 0.4113    | 19.52     |
| 10        | 0.9338    | 0.0       | 5.453     | 0.0       | 21.67     |
{'target': 0.937898616526895, 'params': {'gamma': 0.6817792797809532, 'max_depth': 5.97031132

In [26]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.25, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {


            'max_depth' :(1,10),
            'n_estimators' :(1,50),

            'gamma': (0,1), 
            'min_child_weight' : (0,1)
                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

xgb_model = XGBClassifier(max_depth = int(params['params']['max_depth']),
                                  
                                  n_estimators = int(params['params']['n_estimators']), 
                                  objective = 'binary:logistic',
                                 booster = 'gbtree', 
                                  gamma = params['params']['gamma'], 
                                  min_child_weight = params['params']['min_child_weight'],
                                  #scale_pos_weight= params['params']['scale_pos_weight'],
                                 random_state = 77)

        
tm_st = time.time()
xgb_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = xgb_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


current task: 0 XG
|   iter    |  target   |   gamma   | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9469    | 0.1527    | 3.721     | 0.06204   | 23.53     |
| 2         | 0.9366    | 0.8353    | 9.343     | 0.727     | 38.66     |
| 3         | 0.9041    | 0.2692    | 6.796     | 0.09337   | 4.905     |
| 4         | 0.9421    | 0.5896    | 4.09      | 0.9889    | 31.7      |
| 5         | 0.9338    | 0.6818    | 5.97      | 0.2689    | 19.29     |
| 6         | 0.9241    | 0.0       | 1.0       | 0.0       | 50.0      |
| 7         | 0.9366    | 0.0       | 10.0      | 0.0       | 27.46     |
| 8         | 0.9028    | 1.0       | 1.0       | 1.0       | 26.29     |
| 9         | 0.9317    | 0.6863    | 8.563     | 0.6988    | 19.99     |
| 10        | 0.9338    | 0.2114    | 2.93      | 0.00309   | 23.55     |
{'target': 0.9468965517241379, 'params': {'gamma': 0.152663734901322, 'max_depth': 3.72120948

### the Bayes optimization

In [27]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {


            'max_depth' :(1,10),
            'n_estimators' :(1,50),

            'gamma': (0,1), 
            'min_child_weight' : (0,1)
                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

xgb_model = XGBClassifier(max_depth = int(params['params']['max_depth']),
                                  
                                  n_estimators = int(params['params']['n_estimators']), 
                                  objective = 'binary:logistic',
                                 booster = 'gbtree', 
                                  gamma = params['params']['gamma'], 
                                  min_child_weight = params['params']['min_child_weight'],
                                  #scale_pos_weight= params['params']['scale_pos_weight'],
                                 random_state = 77)

        
tm_st = time.time()
xgb_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = xgb_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
val = cross_val_score(xgb_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()        
print("5-fold Test accuracy: " ,val)          


current task: 0 XG
|   iter    |  target   |   gamma   | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9566    | 0.1527    | 3.721     | 0.06204   | 23.53     |
| 2         | 0.9478    | 0.8353    | 9.343     | 0.727     | 38.66     |
| 3         | 0.9266    | 0.2692    | 6.796     | 0.09337   | 4.905     |
| 4         | 0.9509    | 0.5896    | 4.09      | 0.9889    | 31.7      |
| 5         | 0.9447    | 0.6818    | 5.97      | 0.2689    | 19.29     |
| 6         | 0.9266    | 0.08175   | 6.45      | 0.252     | 4.587     |
| 7         | 0.9369    | 0.0       | 1.0       | 0.0       | 50.0      |
| 8         | 0.9524    | 0.5531    | 9.999     | 0.01329   | 27.86     |
| 9         | 0.9074    | 1.0       | 1.0       | 0.0       | 26.15     |
| 10        | 0.9524    | 0.0       | 5.446     | 0.1005    | 22.6      |
{'target': 0.9565637091483579, 'params': {'gamma': 0.152663734901322, 'max_depth': 3.72120948

### Train on the whole dataset and tune the parameter individually

In [28]:
for para in [10,50,100]:
    
    import sys

    tr_dx = np.array(trainingMat).astype(float)
    tr_dy = np.array(hwLabels).astype(float)
    ts_dx = np.array(testMat).astype(float)
    ts_dy = np.array(tsLabels).astype(float)




    xgb_model = XGBClassifier(

                                      n_estimators = para, 
                                      objective = 'binary:logistic',
                                     booster = 'gbtree', 


                                     random_state = 77)


    tm_st = time.time()
    xgb_model.fit(tr_dx,tr_dy)     
    tm_ed = time.time()
    print("Training:")
    print(tm_ed-tm_st)

    tm_st = time.time()
    test_preds = xgb_model.predict_proba(ts_dx)
    tm_ed = time.time()
    print("Predicting:")
    print(tm_ed-tm_st)


    preds_1 = test_preds.argmax(axis=1)
    ts_accuracy_score = accuracy_score(ts_dy, preds_1)
    print("Test accuracy: " ,ts_accuracy_score)



Training:
33.21349549293518
Predicting:
0.016661405563354492
Test accuracy:  0.9651162790697675
Training:
105.60733532905579
Predicting:
0.020355939865112305
Test accuracy:  0.9704016913319239
Training:
181.02379179000854
Predicting:
0.019367694854736328
Test accuracy:  0.9704016913319239


In [ ]:
for para in [5,10,20]:
    
    import sys

    tr_dx = np.array(trainingMat).astype(float)
    tr_dy = np.array(hwLabels).astype(float)
    ts_dx = np.array(testMat).astype(float)
    ts_dy = np.array(tsLabels).astype(float)




    xgb_model = XGBClassifier(

                                      max_depth = para, 
                                      objective = 'binary:logistic',
                                     booster = 'gbtree', 


                                     random_state = 77)


    tm_st = time.time()
    xgb_model.fit(tr_dx,tr_dy)     
    tm_ed = time.time()
    print("Training:")
    print(tm_ed-tm_st)

    tm_st = time.time()
    test_preds = xgb_model.predict_proba(ts_dx)
    tm_ed = time.time()
    print("Predicting:")
    print(tm_ed-tm_st)


    preds_1 = test_preds.argmax(axis=1)
    ts_accuracy_score = accuracy_score(ts_dy, preds_1)
    print("Test accuracy: " ,ts_accuracy_score)


Training:
144.89028120040894
Predicting:
0.018473148345947266
Test accuracy:  0.9714587737843552


## DT

In [ ]:
def xgb_cv(max_depth):
    
    
    
    xgb_model = DecisionTreeClassifier(max_depth = int(max_depth), 
                              #max_leaf_nodes = int(max_leaf_nodes),
                              
                             )
    
    val = cross_val_score(xgb_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 30).mean()

    return val


def tune( tasks, nb, nit, model_name, param_tune):
    for task in tasks:
        print(f"current task: {task} {model_name}")
        #res_output = f"{task}year_{model_name}.txt"
        #model_dump = f"{task}year_{model_name}_model.pkl"
        


        
        BO = BayesianOptimization( f = xgb_cv,pbounds = param_tune,verbose = 2,random_state = 777 )
        
        BO.maximize(n_iter=5)
        print(BO.max)
        return BO.max


In [ ]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.75, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'max_depth' :(1,20),
            #'min_samples_split' : (2,10),
            #'min_samples_leaf' : (1,100), 
            #'max_leaf_nodes' :(2,100),

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

dt_model = DecisionTreeClassifier(max_depth = int(params['params']['max_depth']),
                                  #min_samples_split = int(params['params']['min_samples_split']), 
                                  #min_samples_leaf = params['params']['min_samples_leaf'], 
                                  #max_leaf_nodes = int(params['params']['max_leaf_nodes']),
                                 random_state = 77)

        
tm_st = time.time()
dt_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = dt_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)
preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        


In [ ]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.5, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'max_depth' :(1,20),
            #'min_samples_split' : (2,10),
            #'min_samples_leaf' : (1,100), 
            #'max_leaf_nodes' :(2,100),

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

dt_model = DecisionTreeClassifier(max_depth = int(params['params']['max_depth']),
                                  #min_samples_split = int(params['params']['min_samples_split']), 
                                  #min_samples_leaf = params['params']['min_samples_leaf'], 
                                  #max_leaf_nodes = int(params['params']['max_leaf_nodes']),
                                 random_state = 77)

        
tm_st = time.time()
dt_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = dt_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)
preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        
        


In [ ]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.25, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'max_depth' :(1,20),
            #'min_samples_split' : (2,10),
            #'min_samples_leaf' : (1,100), 
            #'max_leaf_nodes' :(2,100),

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

dt_model = DecisionTreeClassifier(max_depth = int(params['params']['max_depth']),
                                  #min_samples_split = int(params['params']['min_samples_split']), 
                                  #min_samples_leaf = params['params']['min_samples_leaf'], 
                                  #max_leaf_nodes = int(params['params']['max_leaf_nodes']),
                                 random_state = 77)

        
tm_st = time.time()
dt_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = dt_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)
preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
        
        


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'max_depth' :(1,20),
            #'min_samples_split' : (2,10),
            #'min_samples_leaf' : (1,100), 
            #'max_leaf_nodes' :(2,100),

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

dt_model = DecisionTreeClassifier(max_depth = int(params['params']['max_depth']),
                                  #min_samples_split = int(params['params']['min_samples_split']), 
                                  #min_samples_leaf = params['params']['min_samples_leaf'], 
                                  #max_leaf_nodes = int(params['params']['max_leaf_nodes']),
                                 random_state = 77)

        
tm_st = time.time()
dt_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = dt_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)
preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
val = cross_val_score(dt_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()        
print("5-fold Test accuracy: " ,val)          
        


## KNN

In [ ]:
def xgb_cv(n_neighbors):
    
    
    
    xgb_model = KNeighborsClassifier(n_neighbors = int(n_neighbors),
                              
                             )
    
    val = cross_val_score(xgb_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 30).mean()

    return val


def tune( tasks, nb, nit, model_name, param_tune):
    for task in tasks:
        print(f"current task: {task} {model_name}")
        #res_output = f"{task}year_{model_name}.txt"
        #model_dump = f"{task}year_{model_name}_model.pkl"
        


        
        BO = BayesianOptimization( f = xgb_cv,pbounds = param_tune,verbose = 2,random_state = 777 )
        
        BO.maximize(n_iter=5)
        print(BO.max)
        return BO.max

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.75, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'n_neighbors' :(1,10)

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

kn_model = KNeighborsClassifier(n_neighbors = int(params['params']['n_neighbors']))

        
tm_st = time.time()
kn_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = kn_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.5, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'n_neighbors' :(1,10)

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

kn_model = KNeighborsClassifier(n_neighbors = int(params['params']['n_neighbors']))

        
tm_st = time.time()
kn_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = kn_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.25, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'n_neighbors' :(1,10)

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

kn_model = KNeighborsClassifier(n_neighbors = int(params['params']['n_neighbors']))

        
tm_st = time.time()
kn_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = kn_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




param_tune = {
            'n_neighbors' :(1,10)

                }




tm_st = time.time()
params = tune(tasks = [i], nb = 10, nit = 20, model_name = 'XG', param_tune = param_tune)
tm_ed = time.time()
print("Tuning:")
print(tm_ed-tm_st)

kn_model = KNeighborsClassifier(n_neighbors = int(params['params']['n_neighbors']))

        
tm_st = time.time()
kn_model.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = kn_model.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)

        
val = cross_val_score(kn_model,tr_dx,tr_dy,scoring = 'accuracy',cv = 5,n_jobs = 10).mean()        
print("5-fold Test accuracy: " ,val)          
        


## MLP

In [ ]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.75, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 2, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.01, batch_size = 2")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)

tr_preds = mlp.predict_proba(tr_dx)
preds_tr = tr_preds.argmax(axis=1)
tr_accuracy_score = accuracy_score(tr_dy, preds_tr)
print("Training accuracy: " ,tr_accuracy_score)

preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.5, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 2, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.01, batch_size = 2")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)

tr_preds = mlp.predict_proba(tr_dx)
preds_tr = tr_preds.argmax(axis=1)
tr_accuracy_score = accuracy_score(tr_dy, preds_tr)
print("Training accuracy: " ,tr_accuracy_score)

preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
dtr,_1, ltr,_2 = train_test_split(trainingMat,hwLabels, test_size=0.25, random_state=42)

import sys

tr_dx = np.array(dtr).astype(float)
tr_dy = np.array(ltr).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)




mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 2, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.01, batch_size = 2")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)

tr_preds = mlp.predict_proba(tr_dx)
preds_tr = tr_preds.argmax(axis=1)
tr_accuracy_score = accuracy_score(tr_dy, preds_tr)
print("Training accuracy: " ,tr_accuracy_score)

preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 2, learning_rate_init = 0.01, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.01, batch_size = 2")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)

tr_preds = mlp.predict_proba(tr_dx)
preds_tr = tr_preds.argmax(axis=1)
tr_accuracy_score = accuracy_score(tr_dy, preds_tr)
print("Training accuracy: " ,tr_accuracy_score)

preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 2, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.001, batch_size = 2")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)

tr_preds = mlp.predict_proba(tr_dx)
preds_tr = tr_preds.argmax(axis=1)
tr_accuracy_score = accuracy_score(tr_dy, preds_tr)
print("Training accuracy: " ,tr_accuracy_score)


preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 2, learning_rate_init = 0.0001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.0001, batch_size = 2")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 5, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.001, batch_size = 5")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 10, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.001, batch_size = 10")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)


In [ ]:
import sys

tr_dx = np.array(trainingMat).astype(float)
tr_dy = np.array(hwLabels).astype(float)
ts_dx = np.array(testMat).astype(float)
ts_dy = np.array(tsLabels).astype(float)



mlp = MLPClassifier(activation = "relu",solver = "adam", batch_size = 20, learning_rate_init = 0.001, hidden_layer_sizes=(20, 10), max_iter=200, random_state=42)


mlp.fit(tr_dx, tr_dy)


iterations = range(1, len(mlp.loss_curve_) + 1)
loss_values = mlp.loss_curve_


plt.figure()
plt.title("MLP Training Curve, lr = 0.001, batch_size = 20")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.plot(iterations, loss_values, label="Training Loss")
plt.legend()
plt.show()


tm_st = time.time()
mlp.fit(tr_dx,tr_dy)     
tm_ed = time.time()
print("Training:")
print(tm_ed-tm_st)

tm_st = time.time()
test_preds = mlp.predict_proba(ts_dx)
tm_ed = time.time()
print("Predicting:")
print(tm_ed-tm_st)



preds_1 = test_preds.argmax(axis=1)
ts_accuracy_score = accuracy_score(ts_dy, preds_1)
print("Test accuracy: " ,ts_accuracy_score)



recall = recall_score(ts_dy, preds_1, average='weighted')



precision = precision_score(ts_dy, preds_1, average='weighted')

        
f1_s = f1_score(ts_dy, preds_1, average='weighted')

print("Test recall: ",recall)
print("Test precision: ",precision)        
print("Test f1_s: ",f1_s)
